## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started and install the needed libraries:

In [ ]:
from arcgis.gis import GIS
gis = GIS("home")
import platform
import psutil
import sys
import os
import shutil
import datetime
import time
from datetime import date, timedelta
import pandas as pd
import arcpy
from arcgis.features import GeoAccessor
import arcgis
from arcgis.raster import *
from arcgis.raster.analytics import *

arcgis.env.verbose=True

#### Run this to get the system information

In [ ]:
print("**********Platform information**********")
print(f"System: {platform.system()}")
print(f"Node Name: {platform.node()}")
print(f"Release: {platform.release()}")
print(f"Version: {platform.version()}")
print(f"Machine: {platform.machine()}")
print(f"Processor: {platform.processor()}")
print("Memory information:")
mem = psutil.virtual_memory()
print(f"Total: {mem.total / (1024 ** 3):.2f} GB")
print(f"Available: {mem.available / (1024 ** 3):.2f} GB")
print(f"Used: {mem.used / (1024 ** 3):.2f} GB")
print(f"Percentage: {mem.percent}%")
print("CPU information:")
print(f"Physical cores: {psutil.cpu_count(logical=False)}")
print("**********Platform information**********")

#### Run this to ensure the install the Global Fishing Watch Python API

In [ ]:
gfwapi_path = '/arcgis/home/gfwapiclient'
if not os.path.isdir(gfwapi_path):
    %pip install --target /arcgis/home/gfwapiclient gfw-api-python-client
    # %pip install gfw-api-python-client
    # %pip install --upgrade gfw-api-python-client

In [ ]:
sys.path.append(gfwapi_path)
import gfwapiclient as gfw

#### Load the access credentials for the GFW API

In [ ]:
sys.path.append('/arcgis/home/notebook')
from access_credentials import access_token_key

#### Create a working directory

In [ ]:
if not os.path.exists("/arcgis/home/notebook/gfw"):
    os.makedirs("/arcgis/home/notebook/gfw")

#### Set the working directory as the active workspace

In [ ]:
workspace = "/arcgis/home/notebook/gfw"
arcpy.env.workspace = workspace
arcpy.env.overwriteOutput = True

#### Connect to the GFW Client using the access key

In [ ]:
access_token = os.environ.get(
    "GFW_API_ACCESS_TOKEN",
    access_token_key,
)

gfw_client = gfw.Client(
    access_token=access_token,
)

#### Create a global extent geojson format for use in the GFW client

In [ ]:
global_geojson = {"type": "Polygon", "coordinates": [[[180, 90], [180, -90], [-180, -90], [-180, 90], [180, 90]]]}

#### Create a filegeodatabase in the working directoy to store the features

In [ ]:
gdb_name = "SAR_Infrastructure.gdb"
if not arcpy.Exists(gdb_name):
    arcpy.CreateFileGDB_management(workspace, gdb_name)

#### request the SAR fixed infrastructure data from the GFW client

In [ ]:
sar_infrastructure_result = await gfw_client.datasets.get_sar_fixed_infrastructure(
    geometry=global_geojson
)

#### veiw the response in a dataframe

In [ ]:
sar_infrastructure_df = sar_infrastructure_result.df()


In [ ]:
sar_infrastructure_df

#### Calculate a structure duration from the structure_start_date and the structure_end_date.

In [ ]:
from datetime import datetime, timezone

today_utc = pd.Timestamp(datetime.now(timezone.utc))
sar_infrastructure_df["structure_duration"] = (
    sar_infrastructure_df.apply(
        lambda row: (row["structure_end_date"] - row["structure_start_date"]).days
        if pd.notnull(row["structure_end_date"]) else pd.NA,
        axis=1
    )
)

#### create a spatially enabled dataframe from the dataframe using the long and lat columns.

In [ ]:
sar_infrastructure_sedf = pd.DataFrame.spatial.from_xy(df = sar_infrastructure_df, x_column = "lon", y_column = "lat", sr=4326)

#### Save the SDEF to a feature class in the File Geodatabase

In [ ]:
sar_infrastructure_sedf.spatial.to_featureclass(os.path.join(workspace, gdb_name, "sar_infrastructure"))

#### create a zipped version of the File Geodatabase

In [ ]:
import zipfile

gdb_path = os.path.join(workspace, gdb_name)
zip_path = os.path.join(workspace, f"{gdb_name}.zip")

if not os.path.isdir(gdb_path):
    raise FileNotFoundError(f"File geodatabase not found: {gdb_path}")

# remove existing zip if present
if os.path.exists(zip_path):
    os.remove(zip_path)

# create zip containing the .gdb folder at top-level, excluding any .lock files
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for root, dirs, files in os.walk(gdb_path):
        rel_root = os.path.relpath(root, workspace)
        for fname in files:
            if fname.lower().endswith(".lock"):
                continue
            fullpath = os.path.join(root, fname)
            arcname = os.path.join(rel_root, fname).replace(os.path.sep, "/")
            zf.write(fullpath, arcname=arcname)

print(f"Created zip: {zip_path}")

In [ ]:
SAR_Infrastructure_gdb = gis.content.get("59caea3c970c4932b59bae60738414e4")
SAR_Infrastructure_gdb

In [ ]:
from arcgis.features import FeatureLayerCollection
FL_collection = FeatureLayerCollection.fromitem(SAR_Infrastructure_gdb)

In [ ]:
FL_collection.properties.layers[0].name

In [ ]:
FL_collection.manager.overwrite(zip_path)